## Mount the Drive, and Change to Google Drive Folder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [ ]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist_old = list(pd.read_csv('flist.txt', header = None)[0])
flist = []
change_names = list(map(lambda x: flist.append(f'street_view/{x}'), flist_old))

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [ ]:
properties = pd.read_csv('properties.csv')
properties_juny12 = pd.read_csv('properties_juny12.csv')
properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.head()

Basic information of the dataset is shown as follows.

In [ ]:
properties.info()

Descriptive statistics of continuous variables are shown as follows.

In [ ]:
properties.describe()

Frequencies of each level of the variable `property type` are obtained as follows.

In [ ]:
properties.propertyType.value_counts()

## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [ ]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
change_names_new = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [ ]:
flist_train, flist_test, propertyType_train, propertyType_test = train_test_split(
    flist_new, # image directories
    np.array(properties_sub.propertyType), # property types
    test_size = 0.3)

In [ ]:
Img_list_train = list(map(lambda x: np.asarray(Image.open(x).resize((100, 100), Image.LANCZOS)), flist_train))
Img_list_test = list(map(lambda x: np.asarray(Image.open(x).resize((100, 100), Image.LANCZOS)), flist_test))
Img_array_train = np.asarray(Img_list_train)
Img_array_test = np.asarray(Img_list_test)

For categorical variables, one-hot encoder is introduced.

In [ ]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [ ]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    model.add(Dense(100, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(Img_array_train, dummy_propertyType_train, epochs = 20, batch_size = 20)

In [ ]:
propertyType_test_pred_MLP = np.argmax(MLP.predict(Img_array_test), axis = 1) 
MLP.evaluate(Img_array_test, dummy_propertyType_test)

In [ ]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_MLP)

## Convolutional Neural Network

In [ ]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(5, 2, padding = 'same', activation = tf.nn.leaky_relu))
    model.add(MaxPooling2D())  
    model.add(Flatten())
    model.add(Dense(50, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(Img_array_train, dummy_propertyType_train, epochs = 15, batch_size = 30)

In [ ]:
propertyType_test_pred_CNN = np.argmax(CNN.predict(Img_array_test), axis = 1) 
CNN.evaluate(Img_array_test, dummy_propertyType_test)

In [ ]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_CNN)

In [ ]:
# import dill
# dill.dump_session('Presetting.pkl')
# # dill.load_session('Presetting.pkl')